In [4]:
import pydicom
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import generate_uid
from PIL import Image
import numpy as np
import datetime
import os

def png_to_dcm(png_path, dcm_path):
    # Open the PNG image
    img = Image.open(png_path)
    img = img.convert("RGB")  # Ensure image is in RGB mode
    np_img = np.array(img)

    # Create the DICOM dataset
    file_meta = Dataset()
    file_meta.MediaStorageSOPClassUID = pydicom.uid.SecondaryCaptureImageStorage
    file_meta.MediaStorageSOPInstanceUID = generate_uid()
    file_meta.ImplementationClassUID = generate_uid()

    ds = FileDataset(dcm_path, {}, file_meta=file_meta, preamble=b"\0" * 128)
    
    ds.Modality = 'OT'  # Other
    ds.ContentDate = str(datetime.date.today()).replace('-', '')
    ds.ContentTime = str(datetime.datetime.now().time()).replace(':', '').split('.')[0]

    # These are the necessary imaging components of the file meta-information header
    ds.SOPClassUID = file_meta.MediaStorageSOPClassUID
    ds.SOPInstanceUID = file_meta.MediaStorageSOPInstanceUID
    ds.file_meta = file_meta

    # Add the image data
    ds.Rows, ds.Columns, _ = np_img.shape
    ds.SamplesPerPixel = 3
    ds.PhotometricInterpretation = "RGB"
    ds.PlanarConfiguration = 0
    ds.BitsAllocated = 8
    ds.BitsStored = 8
    ds.HighBit = 7
    ds.PixelRepresentation = 0
    ds.PixelData = np_img.tobytes()

    # Add additional metadata
    ds.PatientName = "Test^Firstname"
    ds.PatientID = "123456"

    # Save the DICOM file
    ds.save_as(dcm_path)
    print(f"DICOM file saved to {dcm_path}")


In [3]:
# Example usage
png_path = 'C:/Users/FDIFRIS/Downloads/my-dicom-image(3).png'
dcm_path = 'C:/Users/FDIFRIS/Downloads/my-dicom-image(3).dcm'
png_to_dcm(png_path, dcm_path)

DICOM file saved to C:/Users/FDIFRIS/Downloads/my-dicom-image(3).dcm
